## PostgreSQL and PostGIS for Nearest Neighbour Query / Proximity Search


Objective is to leverage PostGIS for NNQ.

In [84]:
%pip install --user psycopg2

Note: you may need to restart the kernel to use updated packages.


To enable PostGIS extension, run the following command on database

CREATE EXTENSION postgis;

In [89]:
import psycopg2
from psycopg2 import Error
import pandas as pd

Parsing configuration parameters.

In [90]:
def getConfigOrDefault(config_filename, config,label,default=None):
    if config.get(label) is not None:
        return config.get(label)
    if default is not None:
        return default
    print(f"{config_filename} file does not have {label} parameter!")
    return None

In [91]:
from dotenv import dotenv_values
config_filename = './postgresql.cfg'
config = dotenv_values(config_filename)

USER_NAME = getConfigOrDefault(config_filename, config, 'POSTGRES_UNAME')
USER_PWD  = getConfigOrDefault(config_filename, config, 'POSTGRES_UPWD')
POSTGRES_HOST = getConfigOrDefault(config_filename, config, 'POSTGRES_HOST', 'localhost')
POSTGRES_PORT = getConfigOrDefault(config_filename, config, 'POSTGRES_PORT', '5432')
POSTGRES_DB = getConfigOrDefault(config_filename, config, 'POSTGRES_DBNAME', 'test')

if USER_PWD is None or USER_PWD is None:
    print(f"{config_filename} file does not have parameters: POSTGRES_UNAME and/or POSTGRES_UPWD!")


In [20]:
try:
    # Connect to an existing database
    connection = psycopg2.connect(user=USER_NAME,
                                  password=USER_PWD,
                                  host=POSTGRES_HOST,
                                  port=POSTGRES_PORT,
                                  database=POSTGRES_DB)

    # Create a cursor to perform database operations
    cursor = connection.cursor()
    # Print PostgreSQL details
    print("PostgreSQL server information")
    print(connection.get_dsn_parameters(), "\n")
    # Executing a SQL query
    cursor.execute("SELECT version();")
    # Fetch result
    record = cursor.fetchone()
    print("You are connected to - ", record, "\n")

    #Closing the connection
    connection.close()

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
finally:
    if (connection):
       cursor.close()
       connection.close()
       print("PostgreSQL connection is closed")

PostgreSQL server information
{'user': 'test_user', 'channel_binding': 'prefer', 'dbname': 'postgres', 'host': '127.0.0.1', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslcertmode': 'allow', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'disable', 'krbsrvname': 'postgres', 'gssdelegation': '0', 'target_session_attrs': 'any', 'load_balance_hosts': 'disable'} 

You are connected to -  ('PostgreSQL 17.5 on x86_64-windows, compiled by msvc-19.44.35209, 64-bit',) 

PostgreSQL connection is closed


## Helpers

In [100]:
def execute_sqL_with_fetchnone(sql_stmt):
    """ Executes SQL statement with fetchnone
        Not worried about performance
    """
    print(f"Executing SQL statement with fetchnone-{sql_stmt}")
    try:
        # Connect to an existing database
        connection = psycopg2.connect(user=USER_NAME,
                                  password=USER_PWD,
                                  host=POSTGRES_HOST,
                                  port=POSTGRES_PORT,
                                  database=POSTGRES_DB)

        # Create a cursor to perform database operations
        cursor = connection.cursor()
        # Executing given SQL query
        cursor.execute(sql_stmt)
        connection.commit()
        cursor.close()
        connection.close()

    except (Exception, Error) as error:
        print(f"Error : {error}")
    finally:
        if (connection):
            cursor.close()
            connection.close()
            print(f"Connection to PostgreSQL({POSTGRES_HOST}:{POSTGRES_PORT}) is closed")


In [101]:
def execute_sqL_with_fetchone(sql_stmt):
    """ Executes SQL statement with fetchone
        Not worried about performance
    """
    response = None
    print(f"Executing SQL statement with fetchone-{sql_stmt}")
    try:
        # Connect to an existing database
        connection = psycopg2.connect(user=USER_NAME,
                                  password=USER_PWD,
                                  host=POSTGRES_HOST,
                                  port=POSTGRES_PORT,
                                  database=POSTGRES_DB)

        # Create a cursor to perform database operations
        cursor = connection.cursor()
        # Executing given SQL query
        cursor.execute(sql_stmt)
        # Fetch result so that it can return
        response = cursor.fetchone()

        connection.commit()
        cursor.close()
        connection.close()

    except (Exception, Error) as error:
        print(f"Error : {error}")
    finally:
        if (connection):
            cursor.close()
            connection.close()
            print(f"Connection to PostgreSQL({POSTGRES_HOST}:{POSTGRES_PORT}) is closed")
    return response


In [102]:
def execute_sqL_with_fetchall(sql_stmt):
    """ Executes SQL statement with fetchone
        Not worried about performance
    """
    response = None
    try:
        # Connect to an existing database
        connection = psycopg2.connect(user=USER_NAME,
                                  password=USER_PWD,
                                  host=POSTGRES_HOST,
                                  port=POSTGRES_PORT,
                                  database=POSTGRES_DB)

        # Create a cursor to perform database operations
        cursor = connection.cursor()
        # Executing given SQL query
        cursor.execute(sql_stmt)
        # Fetch result so that it can return
        response = cursor.fetchall()

        connection.commit()
        cursor.close()
        connection.close()

    except (Exception, Error) as error:
        print(f"Error : {error}")
    finally:
        if (connection):
            cursor.close()
            connection.close()
            print(f"Connection to PostgreSQL({POSTGRES_HOST}:{POSTGRES_PORT}) is closed")
    return response


This notebook utilize following helper queries

In [105]:
# Queries
SQL_DROP_TABLE = f"DROP TABLE IF EXISTS test.restaurants;"
SQL_CREATE_TABLE = f"""
    CREATE TABLE IF NOT EXISTS test.restaurants (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255),
    cuisine VARCHAR(64),
    longitude FLOAT,
    latitude FLOAT,
    location GEOMETRY(Point, 4326)
    )
    """
SQL_VALIDATE_COLUMNS="""
     select column_name
        , data_type
        , udt_catalog
        , udt_schema
        , udt_name
        , character_maximum_length
        , column_default
        , is_nullable
    from INFORMATION_SCHEMA.COLUMNS
    where table_name = 'restaurants';
    """

In [104]:
# Drops the restaurants table
reply = execute_sqL_with_fetchnone(SQL_DROP_TABLE)
reply

Executing SQL statement with fetchnone-DROP TABLE IF EXISTS test.restaurants;
Connection to PostgreSQL(127.0.0.1:5432) is closed


In [107]:
# create the restaurants table
reply = execute_sqL_with_fetchnone(SQL_CREATE_TABLE)
reply

Executing SQL statement with fetchnone-
    CREATE TABLE IF NOT EXISTS test.restaurants (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255),
    cuisine VARCHAR(64),
    longitude FLOAT,
    latitude FLOAT,
    location GEOMETRY(Point, 4326)
    )
    
Connection to PostgreSQL(127.0.0.1:5432) is closed


In [108]:
# create the restaurants table
reply = execute_sqL_with_fetchall(SQL_VALIDATE_COLUMNS)
reply

Connection to PostgreSQL(127.0.0.1:5432) is closed


[('id',
  'integer',
  'postgres',
  'pg_catalog',
  'int4',
  None,
  "nextval('test.restaurants_id_seq'::regclass)",
  'NO'),
 ('name',
  'character varying',
  'postgres',
  'pg_catalog',
  'varchar',
  255,
  None,
  'YES'),
 ('cuisine',
  'character varying',
  'postgres',
  'pg_catalog',
  'varchar',
  64,
  None,
  'YES'),
 ('longitude',
  'double precision',
  'postgres',
  'pg_catalog',
  'float8',
  None,
  None,
  'YES'),
 ('latitude',
  'double precision',
  'postgres',
  'pg_catalog',
  'float8',
  None,
  None,
  'YES'),
 ('location',
  'USER-DEFINED',
  'postgres',
  'public',
  'geometry',
  None,
  None,
  'YES')]

Let's check table definition

In [109]:
column_names = [ 'column_name', 'data_type',
                 'udt_catalog', 'udt_schema', 'udt_name',
                 'character_maximum_length', 'column_default', 'is_nullable']
df = pd.DataFrame(reply,columns=column_names)
df

,column_name,data_type,udt_catalog,udt_schema,udt_name,character_maximum_length,column_default,is_nullable
0,id,integer,postgres,pg_catalog,int4,NaN,nextval('test.restaurants_id_seq'::regclass),NO
1,name,character varying,postgres,pg_catalog,varchar,255.0,None,YES
2,cuisine,character varying,postgres,pg_catalog,varchar,64.0,None,YES
3,longitude,double precision,postgres,pg_catalog,float8,NaN,None,YES
4,latitude,double precision,postgres,pg_catalog,float8,NaN,None,YES
5,location,USER-DEFINED,postgres,public,geometry,NaN,None,YES


In [110]:
filename="./restaurants.csv"
df_loaded = pd.read_csv(filename)
df_loaded[:10]

,Name,Lon,Lat
0,Morris Park Bake Shop,-73.856077,40.848447
1,Wendy'S,-73.961704,40.662942
2,Riviera Caterer,-73.982420,40.579505
3,Tov Kosher Kitchen,-73.860115,40.731174
4,Brunos On The Boulevard,-73.880383,40.764312
5,Dj Reynolds Pub And Restaurant,-73.985136,40.767692
6,Wilken'S Fine Food,-73.906851,40.619903
7,Regina Caterers,-74.005289,40.628886
8,Taste The Tropics Ice Cream,-73.948261,40.640827
9,Kosher Island,-74.137729,40.611957


In [111]:
df_loaded['Rid'] = df_loaded.index
printdf = df_loaded[:10]
print(printdf.to_string(index=False))

                          Name        Lon       Lat  Rid
         Morris Park Bake Shop -73.856077 40.848447    0
                       Wendy'S -73.961704 40.662942    1
               Riviera Caterer -73.982420 40.579505    2
            Tov Kosher Kitchen -73.860115 40.731174    3
       Brunos On The Boulevard -73.880383 40.764312    4
Dj Reynolds Pub And Restaurant -73.985136 40.767692    5
            Wilken'S Fine Food -73.906851 40.619903    6
               Regina Caterers -74.005289 40.628886    7
   Taste The Tropics Ice Cream -73.948261 40.640827    8
                 Kosher Island -74.137729 40.611957    9


In [112]:
import random
def assignCuisineRandomly(df,cuisine,cname):
    for i in range(df.shape[0]):
        df.at[i,cname] = cuisine[random.randint(0,len(cuisine)-1)]

In [113]:
cuisine = ['italian', 'chinese', 'french', 'zambian', 'egyptian', 'canadian', 'mexican', 'vietnamese', 'cajun', 'korean', 'thai', 'brazilian','colombian','peruvian','ecuadorian', 'japanese','indian','malaysian','russian', 'indonesian']
assignCuisineRandomly(df_loaded,cuisine,'Cuisine')
df_loaded[-4:]

,Name,Lon,Lat,Rid,Cuisine
4996,Wagner College - Hawk' Nest,-74.092853,40.615121,4996,egyptian
4997,Ellen Deli & Grocery,-74.007810,40.725708,4997,malaysian
4998,Crepes On Columbus,-73.961831,40.801052,4998,mexican
4999,Capital Grille,-73.974723,40.751244,4999,japanese


In [116]:
def formatSQL(df):
    name = df['Name'].replace("'"," ")
    sql_stmt = f"INSERT INTO test.restaurants (name, cuisine, longitude, latitude, location) VALUES ('{name}','{df['Cuisine']}',{df['Lon']},{df['Lat']},ST_GeomFromText('POINT({df['Lon']} {df['Lat']})', 4326));"

    return sql_stmt

In [117]:
#print(formatSQL(df_loaded[0]))
idx=4996
print(type(df_loaded.loc[idx].to_dict()))
df_loaded.loc[idx].to_dict()
print(formatSQL(df_loaded.loc[idx].to_dict()))

<class 'dict'>
INSERT INTO test.restaurants (name, cuisine, longitude, latitude, location) VALUES ('Wagner College - Hawk  Nest','egyptian',-74.09285299999999,40.61512099999999,ST_GeomFromText('POINT(-74.09285299999999 40.61512099999999)', 4326));


Let's load the data

In [118]:
# Load data to test.restaurants table
# using ST_GeomFromText('POINT($longitude $latitude)', 4326)
try:
        # Connect to an existing database
        connection = psycopg2.connect(user=USER_NAME,
                                      password=USER_PWD,
                                      host=POSTGRES_HOST,
                                      port=POSTGRES_PORT,
                                      database=POSTGRES_DB)

        # Create a cursor to perform database operations
        cursor = connection.cursor()
        # Executing a SQL query
        cursor.execute("SELECT version();")
        # Fetch result
        record = cursor.fetchone()
        print("You are connected to - ", record, "\n")
        sql_stmt = ""
        for i in range(df_loaded.shape[0]):
            #addARestaurant(connection, df_loaded.loc[i].to_dict())
            sql_stmt = formatSQL(df_loaded.loc[i].to_dict())
            #print(sql_stmt)
            cursor.execute(sql_stmt)

        connection.commit()

        #Closing the connection
        connection.close()

except (Exception, Error) as error:
        print("Error: ", error)
finally:
        if (connection):
            cursor.close()
            connection.close()
        print("PostgreSQL connection is closed")

You are connected to -  ('PostgreSQL 17.5 on x86_64-windows, compiled by msvc-19.44.35209, 64-bit',) 

PostgreSQL connection is closed


Let's search

In [129]:
NNQ_LON = -74.092853
NNQ_LAT =  40.615121
NNQ_RADIUS = 2000/10000
query = f"""
SELECT name
    , cuisine
    , longitude
    , latitude
    , ST_Distance(location, ST_Point({NNQ_LON}, {NNQ_LAT},4326)) AS dist2NNQ
FROM test.restaurants
WHERE ST_DWithin(location, ST_Point({NNQ_LON}, {NNQ_LAT},4326), {NNQ_RADIUS});
"""

explain_query = "Explain "+ query

In [127]:
reply = execute_sqL_with_fetchall(query)
reply

Connection to PostgreSQL(127.0.0.1:5432) is closed


[('Wendy S', 'egyptian', -73.961704, 40.662942, 0.13959551655408542),
 ('Riviera Caterer',
  'brazilian',
  -73.98241999999999,
  40.579505,
  0.11603424901727748),
 ('Dj Reynolds Pub And Restaurant',
  'indonesian',
  -73.98513559999999,
  40.7676919,
  0.18676433757431565),
 ('Wilken S Fine Food',
  'zambian',
  -73.9068506,
  40.6199034,
  0.1860638711720318),
 ('Regina Caterers',
  'indonesian',
  -74.00528899999999,
  40.628886,
  0.08863932152832932),
 ('Taste The Tropics Ice Cream',
  'zambian',
  -73.9482609,
  40.6408271,
  0.14685938498993956),
 ('Kosher Island', 'vietnamese', -74.1377286, 40.6119572, 0.04498698818324906),
 ('C & C Catering Service',
  'vietnamese',
  -73.9973325,
  40.61174889999999,
  0.09558000302710451),
 ('1 East 66Th Street Kitchen',
  'zambian',
  -73.96926909999999,
  40.7685235,
  0.19699062755741206),
 ('Seuda Foods', 'zambian', -73.9653967, 40.6064339, 0.12775200239566312),
 ('Carvel Ice Cream',
  'chinese',
  -73.97822040000001,
  40.6435254,
  0.

In [128]:
column_names = [ 'name', 'cuisine','longitude', 'latitude', 'dist2NNQ']
df = pd.DataFrame(reply,columns=column_names)
df

,name,cuisine,longitude,latitude,dist2NNQ
0,Wendy S,egyptian,-73.961704,40.662942,1.395955e-01
1,Riviera Caterer,brazilian,-73.982420,40.579505,1.160342e-01
2,Dj Reynolds Pub And Restaurant,indonesian,-73.985136,40.767692,1.867643e-01
3,Wilken S Fine Food,zambian,-73.906851,40.619903,1.860639e-01
4,Regina Caterers,indonesian,-74.005289,40.628886,8.863932e-02
...,...,...,...,...,...
3103,Cafe Exchange,thai,-74.013321,40.706824,1.213863e-01
3104,1 Darbar,egyptian,-73.973526,40.753517,1.827355e-01
3105,Wagner College - Hawk Nest,egyptian,-74.092853,40.615121,2.009718e-14
3106,Ellen Deli & Grocery,malaysian,-74.007810,40.725708,1.395055e-01


Let's check the query plan

In [130]:
reply = execute_sqL_with_fetchall(explain_query)
reply

Connection to PostgreSQL(127.0.0.1:5432) is closed


[('Seq Scan on restaurants  (cost=0.00..62630.50 rows=1 width=49)',),
 ("  Filter: st_dwithin(location, '0101000020E61000007995B54DF18552C09A0AF148BC4E4440'::geometry, '0.2'::double precision)",)]

It scans the whole table.